# Imports and utility functions

In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

In [2]:
from pywikiapi import Site

site = Site('http://wikipast.epfl.ch/wikipast/api.php') # Définition de l'adresse de l'API
site.no_ssl = True # Désactivation du https, car pas activé sur wikipast
site.login("MaximeJan@BottinBot3", "nngrqgt0bmo53ogbtie45jjgem7g0tnv") # Login du bot


In [3]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [4]:
import pandas as pd

from IPython.display import display
from urllib.parse import quote

doc2start = {"bpt6k63243601": 123, "bpt6k62931221": 151, "bpt6k6286466w": 189, "bpt6k6393838j": 219, "bpt6k6331310g": 216, "bpt6k6292987t": 353, "bpt6k62906378": 288, "bpt6k6391515w": 319, "bpt6k6315927h": 349, "bpt6k6319106t": 324, "bpt6k6315985z": 82, "bpt6k63959929": 82, "bpt6k63197984": 56, "bpt6k6389871r": 77, "bpt6k6319811j": 79, "bpt6k6282019m": 72, "bpt6k6314752k": 190, "bpt6k6305463c": 113, "bpt6k6318531z": 108, "bpt6k6324389h": 72, "bpt6k63243920": 80, "bpt6k6309075f": 96, "bpt6k6333200c": 132, "bpt6k63243905": 134, "bpt6k6333170p": 137, "bpt6k96727875": 135, "bpt6k9764746t": 99, "bpt6k97645375": 123, "bpt6k9672117f": 125, "bpt6k9763554c": 123, "bpt6k9763553z": 105, "bpt6k9677392n": 110, "bpt6k9692809v": 113, "bpt6k9762929c": 129, "bpt6k9672776c": 119, "bpt6k9764647w": 121, "bpt6k9669143t": 145, "bpt6k9677737t": 139, "bpt6k9668037f": 167, "bpt6k96839542": 171, "bpt6k96762564": 185, "bpt6k9685861g": 189, "bpt6k9763471j": 153, "bpt6k9762899p": 157, "bpt6k97630871": 11, "bpt6k9684454n": 235, "bpt6k9732740w": 239, "bpt6k9684013b": 189, "bpt6k9692626p": 305, "bpt6k9685098r": 281, "bpt6k9764402m": 329, "bpt6k97631451": 322, "bpt6k9776121t": 49, "bpt6k9775724t": 33, "bpt6k97774838": 327, "bpt6k9780089g": 339}

def entry2url(row):
    """
    Takes a row of an Annuaire csv and
    transforms it to the corresponding Gallica url
    """
    url = "https://gallica.bnf.fr/ark:/12148/"
    
    directory = row['directory']
    page = row['page'] - doc2start[directory]
    url += f"{row['directory']}/f{row['page']-doc2start[row['directory']]}"
    
    r_strings = []
    if 'name' in row and pd.notna(row['name']):
        r_strings.append(quote(row['name'].replace('.', ' ')))
    if 'job' in row and pd.notna(row['job']):
        r_strings.append(quote(row['job'].replace('.', ' ')))
    if 'street' in row and pd.notna(row['street']):
        r_strings.append(quote(row['street'].replace('.', ' ')))
    if 'number' in row and pd.notna(row['number']):
        r_strings.append(quote(row['number'].replace('.', ' ')))
    
    if len(r_strings) > 0:
        url += f".item.r={'%20'.join(r_strings)}.zoom"
    
    return url

def add_clickable_url(bottin_dataframe):
    bottin_dataframe = bottin_dataframe.copy()
    bottin_dataframe['url'] = bottin_dataframe.apply(entry2url, axis=1)
    def make_clickable(val):
        return val

    return bottin_dataframe

# Read data from CSV

In [5]:
data_bottin = pd.read_csv('./bottin_data_groupe_3.zip')
data_bottin = add_clickable_url(data_bottin[:])

In [6]:
data_per_years = []
for year in range(1857, 1865):
    data_per_years.append(data_bottin[data_bottin['year'] == year])

# Fill the "entry" dictionnary with all distinct people in the Bottin

In [7]:
entries = {}
for index, row in data_per_years[0].iterrows():
    if row['name'] in entries:
        entries[row['name']].append([(str(row['job']), str(row['street_clean'])), [index]])
    else:
        entries[row['name']] = [[(str(row['job']), str(row['street_clean'])), [index]]]

coef = 0.85
for i in range(1, len(data_per_years)):
    for index, row in data_per_years[i].iterrows():
        if row['name'] in entries:
            found = False
            for person in entries[row['name']]:
                if similar(person[0][0],str(row['job'])) >= coef and similar(person[0][1], str(row['street_clean'])) >= coef:
                    person[1].append(index)
                    found=True
                    break
                elif similar(person[0][0],str(row['job'])) >= coef or similar(person[0][1], str(row['street_clean'])) >= coef:
                    person[0] = (str(row['job']), str(row['street_clean']))
                    person[1].append(index)
                    found=True
                    break
            if not found:
                entries[row['name']].append([(str(row['job']), str(row['street_clean'])), [index]])
        else:
            entries[row['name']] = [[(str(row['job']), str(row['street_clean'])), [index]]]

In [8]:
print(entries['Ollivier'])

[[('café de la Redoute', 'Grenelle-SaintHonoré'), [47444]], [('limonadier', 'boulevard Beaumarchais'), [47447, 115747, 530253, 313487]], [('monteur en bronzes', 'Saintonge'), [47448, 180286, 241816, 383918, 115748, 530255, 313489, 455946]], [('rentier', 'Larochefoucauld'), [47450]], [("premier commis à la direction de l'enregistr", 'et du timbre'), [180283, 241813]], [("grand café de l'Univers et de la Comédie-Française", 'Rohan'), [241812]], [('crémier', 'Oratoire-du-Louvre'), [241815]], [('peintre-artiste', 'Sus-Pères'), [383920, 115750, 530257, 313491, 455948]], [('tailleur', 'Madeleine'), [115751, 530258, 313492, 455949]], [('vins en gros', 'Port-de-Bercy'), [115752, 530259, 313493, 455950]], [('greffier au tribunal de lre instance', 'Rivoli'), [313486, 455942]], [('papiers peints', 'Grande-Rue-Passy'), [313490, 455947]], [('fab. de fautenils', 'Keller'), [455941]], [('herboriste', 'Laval'), [455943]], [('médecin', 'Pernelle'), [455945]]]


# Upload the data to Wikipast

In [13]:
for k,v in entries.items():
    appearences = []
    for appearence in v:
        appearences.append(data_bottin.iloc[appearence[1],:])
    appearences_data = []

    for i in range(len(appearences)):
        appearences_data.append([])
        for index,row in appearences[i].iterrows():
            appearences_data[i].append((str(row['year']), str(row['name']), str(row['job']), str(row['street_clean']), str(row['number']), str(row['url'])))
        appearences_data[i] = sorted(appearences_data[i])


    for i in range(len(appearences_data)):
        entry_string = ""
        for year, name, job, street, nb, url in appearences_data[i]:
            entry_string += f'*[[{year}]] / [[Paris]], [[{street}]] {nb} Mention de {name} avec la catégorie {job} [{url}]\n'
        if(len(appearences_data)>1):
            title = name + " "+job
        else:
            title = name
        try:
            site('edit', title=title,
             text=entry_string,
             token=site.token(),
              createonly=True)
        except:
            if len(appearences_data) == 1:
                title += " "+job
            else:
                title += " "+street
            try:
                site('edit', title=title,
                     text=entry_string,
                     token=site.token(),
                      createonly=True)
            except:
                if len(appearences_data) == 1:
                    title += " "+street
                else:
                    continue
                try:
                    site('edit', title=title,
                         text=entry_string,
                         token=site.token(),
                          createonly=True)
                except:
                    pass

# Create dictionary with all the streets and their inhabitants

In [20]:
streets = {}
for index, row in data_per_years[0].iterrows():
    if row['street_clean'] in streets:
        streets[row['street_clean']].append(index)
    else:
        streets[row['street_clean']] = [index]


coef = 0.85
for i in range(1, len(data_per_years)):
    for index, row in data_per_years[i].iterrows():
        if row['street_clean'] in streets:
            found = False
            for person in streets[row['street_clean']]:
                if similar(person[0],str(row['street_clean'])) >= coef:
                    person[1].append(index)
                    found=True
                    break
                elif similar(person[0][0],str(row['job'])) >= coef or similar(person[0][1], str(row['street_clean'])) >= coef:
                    person[0] = (str(row['job']), str(row['street_clean']))
                    person[1].append(index)
                    found=True
                    break
            if not found:
                streets[row['street_clean']].append([str(row['name']), [index]])
            

'"coef = 0.85\nfor i in range(1, len(data_per_years)):\n    for index, row in data_per_years[i].iterrows():\n        if row[\'street_clean\'] in streets:\n            found = False\n            for person in streets[row[\'street_clean\']]:\n                if similar(person[0][0],str(row[\'job\'])) >= coef and similar(person[0][1], str(row[\'street_clean\'])) >= coef:\n                    person[1].append(index)\n                    found=True\n                    break\n                elif similar(person[0][0],str(row[\'job\'])) >= coef or similar(person[0][1], str(row[\'street_clean\'])) >= coef:\n                    person[0] = (str(row[\'job\']), str(row[\'street_clean\']))\n                    person[1].append(index)\n                    found=True\n                    break\n            if not found:\n                streets[row[\'street_clean\']].append([str(row[\'name\']), [index]])\n                '

In [24]:
print(streets['passage Choiseul'])

[['Aaron', [0]], ['Béronie', [4535]], ['Blanche', [5730]], ['Bréauté (J.)', [8490]], ['Chabonne', [11027]], ['Charpentier', [11770]], ['Cochegrus (Charles)', [13256]], ['Courieux', [14563]], ['Crin', [14985]], ['Darbo', [15630]], ['David', [15930]], ['Decourt', [16347]], ['Delacroix', [16744]], ['Derouart', [19005]], ['Duval', [21897]], ['François (L.)', [24702]], ['Gaissard', [25310]], ['Gentil *', [26496]], ['Girard', [27217]], ['Godet', [27640]], ['Henry (Th.)', [30840]], ['Himmes', [31311]], ['Huard (Mme)', [31701]], ['Jacob', [32327]], ['Jeanne (A.)', [32906]], ['Lacroir ( G: ;', [34542]], ['Lami', [35204]], ['Laroche (Mme)', [35843]], ['Lebreton', [36804]], ['Léonard-Schueitz', [38991]], ['Levot', [40007]], ['Malpel', [41758]], ['Marchi (Salvatore)', [42204]], ['Margue', [42341]], ['Maujard (N.)', [43552]], ['Meyer S. J.)', [44514]], ['Michel', [44650]], ['Pecquet', [48483]], ['Percepied', [48765]], ['Peton', [49435]], ['Philip', [49557]], ['Pierrard', [49890]], ['Pillet', [50060

# Upload streets to Wikipast

In [76]:
import urllib
http = urllib3.PoolManager()
for street,v in streets.items():
    print(street)
    persons = []
    for person in v:
        persons.append(data_bottin.iloc[person[1][0],[4,5]]) #appearences contient toute la donnée sur le gars
    
    entry_string = ""
    for i in range(len(persons)):
        name,job = persons[i]
        r = http.request('GET', f'http://wikipast.epfl.ch/wiki/{name}')
        if(street in str(r.data)):
            entry_string += f'*[[{name}]]\n'
        """else:
            r = http.request('GET', f'http://wikipast.epfl.ch/wiki/{name}{job}') ##A modifier peut-être!
            if (street in str(r.data)):
                entry_string += f'*[[{name} {job}]]"""
            
    try:
        print(entry_string)
        site('edit', title=street,
        text=entry_string,
        token=site.token())
    except:
        print(f"Problème dans l'ajout de l'entrée à cause de la rue {street}")
    break

passage Choiseul
*[[Aaron]]



In [48]:
import urllib3

In [54]:
http = urllib3.PoolManager()
r = http.request('GET', 'http://wikipast.epfl.ch/wiki/Aaron')

In [61]:
'Bondy' in str(r.data)
    

False

In [63]:
data_bottin.head(20)

,directory,page,row,year,name,job,street,number,street_clean,street_only,url
0,bpt6k6305463c,226,0,1857,Aaron,bronzes,passage Choiseul,72 et 74.,passage Choiseul,Choiseul,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
1,bpt6k6305463c,226,1,1857,Aaron (Michel),manuf. de porcelaines,Bondy,30.,Bondy,Bondy,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
2,bpt6k6305463c,226,3,1857,Abadie,architecte,Provence,7.,Provence,Provence,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
3,bpt6k6305463c,226,6,1857,Abard (Mme),corsets,boul. St-Denis,1.,boulevard Saint-Denis,Saint-Denis,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
4,bpt6k6305463c,226,7,1857,Abat,rentier,Isly,10.,Isly,Isly,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
5,bpt6k6305463c,226,14,1857,Abault et Coudray,charpentiers,Corbeau,23.,Corbeau,Corbeau,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
6,bpt6k6305463c,226,15,1857,Abault (Paul),libraire,quai des Augustins,9,quai des Augustins,s Augustins,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
7,bpt6k6305463c,226,16,1857,Abavid,vins,Beaujolais-du-Temple,7.,Beaujolais-du-Temple,Beaujolais-du-Temple,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
8,bpt6k6305463c,226,17,1857,Abazaer (Are),cristaux et porcelaines,Enghien,43.,Enghien,Enghien,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
9,bpt6k6305463c,226,21,1857,Abbattie (Mme),sage-femme,Rivoli,87.,Rivoli,Rivoli,https://gallica.bnf.fr/ark:/12148/bpt6k6305463...
